In [1]:
import pickle

movie = pickle.load( open( "movie_tweet_2.pkl", "rb" ) )

In [2]:
from copy import copy

movie_tweet = copy(movie)

In [3]:
import re

for idx, data in enumerate(movie_tweet):
    title = data['title'].lower()
    movie_conversation = []
    for tweet in data['tweet']:
        cleaned_tweet = re.sub('[^A-Za-z0-9_\s]', "", tweet.lower())
        tweet_list = cleaned_tweet.split(" ")
        if title in tweet_list:
            tweet_list.remove(title)
        if '' in tweet_list:
            space_count = tweet_list.count('')
            for _ in range(space_count):
                tweet_list.remove('')
    
        movie_conversation.append(' '.join(tweet_list))
    movie_tweet[idx]['tweet'] = movie_conversation

In [4]:
from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

st = PorterStemmer()
stop = set(stopwords.words('english'))

tweet_token = copy(movie_tweet)

for idx, data in enumerate(movie_tweet):
    tweet_data = []
    for tweet in data['tweet']:
#         tweet_data.append(pos_tag(word_tokenize(tweet)))
        token_and_stem = [st.stem(i) for i in word_tokenize(tweet)]
        filtered_words = [i for i in token_and_stem if i not in stop]    
        
        tweet_data.append(filtered_words)
        
    tweet_token[idx]['tweet'] = tweet_data

In [5]:
import pandas as pd

df = pd.DataFrame.from_dict(tweet_token)

In [6]:
vocab = []
for i in df.tweet:
    for j in i:
        vocab += j

In [7]:
import collections

#print(collections.Counter(vocab).most_common(3000), "\n")
most_common_vocab = collections.Counter(vocab).most_common(5000)

_vocab = [word for word, common in most_common_vocab]

unknown_token = 'UNKNOWN_TOKEN'
_vocab.append(unknown_token)

In [8]:
vocab_dict = {word:idx for idx, word in enumerate(_vocab)}

In [9]:
index_to_word = copy(df.tweet)

index_to_word[0]

[['made', 'ref', 'help', 'peopl', 'draw', 'fireemblemhero'],
 ['ref', 'use', 'make', 'tutori'],
 ['goddess', 'thank', 'yiu', 'much'],
 ['np', 'gotchu', '3']]

In [10]:
for row, thread in enumerate(index_to_word):
    for num ,tweet in enumerate(thread):
        conversation_list = []
        for idx, word in enumerate(tweet):
#             one_hot_tweet  = [0 for _ in range(len(vocab_dict))]
#             print(vocab_dict[word])
#             one_hot_tweet[vocab_dict[word]] = 1
#         conversation_list.append(one_hot_tweet)
            if word in vocab_dict.keys():
                index_to_word[row][num][idx] = vocab_dict[word]
            else:
                index_to_word[row][num][idx] = vocab_dict[unknown_token]

In [11]:
title_index = copy(df.title)

In [12]:
title_dict = {title:idx for idx, title in enumerate(set(title_index))}

In [13]:
for idx, title in enumerate(title_index):
    title_index[idx] = title_dict[title]

In [14]:
for a, b in title_dict.items():
    if b==1536:
        print(a)

Commando


In [15]:
len(title_dict)

1847

In [16]:
len(title_index)

7919

In [17]:
max_word_size = 0
max_conversation_size = 0

for conversation_list in index_to_word:
    if len(conversation_list) > max_conversation_size:
        max_conversation_size = len(conversation_list)
    for word_list in conversation_list:
        if len(word_list) > max_word_size:
            max_word_size = len(word_list)

In [18]:
max_conversation_size, max_word_size

(9, 27)

In [19]:
DEFAULT_VALUE = 10000

for conversation_idx, conversation_list in enumerate(index_to_word):
    if len(conversation_list) < max_conversation_size:
        default_list_size = max_conversation_size-len(conversation_list)
        for _ in range(default_list_size):
            default_list = [DEFAULT_VALUE for i in range(max_word_size)]
            index_to_word[conversation_idx].append(default_list)
            
    for word_list_idx, word_list in enumerate(conversation_list):
        if len(word_list) < max_word_size:
            current_word_list_size = len(word_list)
            empty_word_space_size = max_word_size - current_word_list_size
            for _ in range(empty_word_space_size):
                index_to_word[conversation_idx][word_list_idx].append(DEFAULT_VALUE)

In [20]:
x_data = []
for i in index_to_word:
    row_data = []
    for j in i:
        row_data+= j
    x_data.append(row_data)

In [21]:
import tensorflow as tf
import numpy as np

In [22]:
n_classes = len(title_dict)

#X = tf.placeholder(tf.float32, [None, max_conversation_size, max_word_size])
X = tf.placeholder(tf.float32, [None, max_conversation_size])
Y = tf.placeholder(tf.int32, [None, n_classes])
# Y = tf.placeholder(tf.int32, [n_classes])

rnn_size = n_classes
num_layers = 4
time_step_size = max_conversation_size
batch_size = 1


lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*num_layers)
state = tf.zeros([batch_size, cell.state_size])
X_split = tf.split(0, time_step_size, X)
outputs, state = tf.nn.rnn(cell, X_split , state)

In [23]:
print(outputs)
print("\n")
print(state)

[<tf.Tensor 'RNN/MultiRNNCell/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_1/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_2/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_3/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_4/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_5/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_6/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_7/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_8/Cell3/BasicLSTMCell/mul_2:0' shape=(1, 1847) dtype=float32>]


Tensor("RNN/concat_8:0", shape=(1, 14776), dtype=float32)


In [24]:
logits = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
targets = tf.reshape(Y, [-1])
weights = tf.ones(time_step_size*batch_size)

In [25]:
loss = tf.nn.seq2seq.sequence_loss_by_example(
    [logits], [targets],[weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.AdamOptimizer(0.01, 0.9).minimize(cost)

In [26]:
x_data = tf.Variable(x_data)
y_data = tf.reshape(tf.reshape(title_index, [-1]))

with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    for i in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    result = sess.run(train_op, feed_dict={X: x_data, Y: y_data})

ValueError: Argument must be a dense tensor: 0        837
1        837
2        837
3        837
4        837
5        837
6        837
7        837
8        837
9        837
10       837
11       837
12       837
13       837
14       837
15         4
16       762
17       467
18       467
19       467
20       467
21       467
22       467
23       467
24       916
25       198
26      1338
27      1338
28      1338
29      1338
        ... 
7889     194
7890     194
7891     194
7892     194
7893    1358
7894    1358
7895    1358
7896    1358
7897     994
7898     994
7899     994
7900     994
7901     994
7902     994
7903     994
7904     994
7905     994
7906     994
7907     152
7908     152
7909     152
7910     519
7911     519
7912     618
7913    1418
7914    1418
7915    1418
7916    1418
7917    1418
7918    1418
Name: title, dtype: object - got shape [7919], but wanted [].